In [4]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.0 pyspark-shell'


import sys
import time
import json
import geohash as gh
from pymongo import MongoClient
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

def sendDataToDB(iter):
    client = MongoClient()
    db = client.fit5148_assignment_db
    streaming_data = db.StreamingData
    sender_1=[]
    sender_2, sender_3 = {},{}
    for record in iter:
            #change record data into json type
            data = json.loads(record[1])
            #add new attribute as geohash, and do the geohasing for each data
            data['geohash']= gh.encode(float(data['latitude']),float(data['longitude']),precision=5)
            print(data)
            #recoggnize the data from which producer, and save into related sender variable
            if(data['sender_id']=='messenger_1'): 
                data['hotspot'] = ""
                sender_1.append(data)
            elif(data['sender_id']=='messenger_2'):
                sender_2= data
            else:
                sender_3= data
    #if sender2 and sender3 all got the data from producer            
    if any(sender_2) and any(sender_3):
        #if these two at the same location
        if(sender_2['geohash']==sender_3['geohash']):
            #calculate the average of surface temperature and confidence of two hotspot, and save it into sender2 var 
            sender_2['surface_temperature_celcius']=\
            int(sender_2['surface_temperature_celcius'])+int(sender_3['surface_temperature_celcius'])/2
            sender_2['confidence']=int(sender_2['confidence'])+int(sender_3['confidence'])/2
            #set the sender3 as empty
            sender_3={}
            
    #get climate data and compare to hotspot data    
    for senderdata in sender_1: 
        #if only have sender2 data
        if any(sender_2):
            #check the location is same or not
            if(senderdata['geohash']==sender_2['geohash']):
                del sender_2['latitude']
                del sender_2['longitude']
                #if is the same, save the hotspot into climate data
                senderdata['hotspot']=sender_2
        #if only have sender3 data
        if any(sender_3):
            #check the location is same or not
            if(senderdata['geohash']==sender_3['geohash']):
                del sender_3['latitude']
                del sender_3['longitude']
                #if is the same, save the hotspot into climate data
                senderdata['hotspot']=sender_3

        try:
            streaming_data.insert(senderdata)
        except Exception as ex:
            print("Exception Occured. Message: {0}".format(str(ex)))
    client.close()

n_secs = 10
topic = "Climate_Hotpot_Streaming"

conf = SparkConf().setAppName("KafkaStreamProcessor").setMaster("local[2]")
sc = SparkContext.getOrCreate()
if sc is None:
    sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, n_secs)
    
kafkaStream = KafkaUtils.createDirectStream(ssc, [topic], {
                        'bootstrap.servers':'127.0.0.1:9092', 
                        'group.id':'week12-group', 
                        'fetch.message.max.bytes':'15728640',
                        'auto.offset.reset':'largest'})
                        # Group ID is completely arbitrary

lines = kafkaStream.foreachRDD(lambda rdd: rdd.foreachPartition(sendDataToDB))

ssc.start()
try:
    ssc.awaitTermination(timeout=600)
except KeyboardInterrupt:
    ssc.stop()
    sc.stop()
    
ssc.stop()
sc.stop()
    
# ssc.stop(stopSparkContext=True,stopGraceFully=True)

In [3]:
client = MongoClient()
db = client.fit5148_assignment_db
streaming_data = db.StreamingData
streaming_data.drop()